In [2]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import pandas as pd
modelname = "medicalai/ClinicalBERT"
bertTokenizer = AutoTokenizer.from_pretrained(modelname)
bertModel = AutoModel.from_pretrained(modelname)
#https://huggingface.co/medicalai/ClinicalBERT
dataset = pd.read_csv('fullembeddings.csv')
tensor_data = torch.tensor(dataset.values)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/var/folders/4g/khtx1g1d45b0d9g7569zqq2r0000gn/T/ipykernel_16269/55445840.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pan

In [3]:
print(tensor_data.dtype)

torch.float64


In [4]:
def calculate_similarity (tensor_a, tensor_b):
  # Calculate the dot product of the two tensors
  dot_product = torch.dot(tensor_a, tensor_b)

  # Calculate the magnitudes of the two tensors
  magnitude_a = torch.norm(tensor_a)
  magnitude_b = torch.norm(tensor_b)

  # Calculate the cosine similarity
  cosine_similarity = dot_product / (magnitude_a * magnitude_b)

  # Print the cosine similarity
  return(cosine_similarity)

In [10]:
closestidx = 0
closeness = 0
questions = {58: "What role do transcription factors play in regulating the activity of RNA polymerase II during gene transcription?",
             63: "What types of early-onset cancers are individuals with Li-Fraumeni syndrome at an increased risk for due to heterozygous loss-of-function mutations in the gene encoding p53?",
             111: "Which amino acids are conditionally essential, and what are the specific precursor amino acids needed for the synthesis of cysteine, tyrosine, arginine, proline, and glycine?",
             218: "What is the mechanism of toxicity for organophosphorus (OP) compounds, and how does the rate of ageing of the OP-enzyme complex influence the severity of poisoning?",
             260: "How do organophosphorus (OP) compounds cause toxicity, what role does the rate of ageing of the OP-enzyme complex play in determining the severity of poisoning, and how do different compounds such as dimethyl, diethyl, and nerve agents vary in their ageing rates?",
             269: "What are the steps involved in the diagnosis and initial medical management of severe hypercalcaemia, including the role of intravenous saline, bisphosphonates, and calcitonin, and what further actions should be taken if hypercalcaemia persists or indicates tumor progression?",
             322: "What are the typical infectious diseases causing unexplained fever among travellers returning from tropical regions, and how does the evolving global travel pattern influence the epidemiology of infectious diseases? Additionally, how effective are pre-travel interventions such as vaccination, insect bite prevention, and food and water hygiene in mitigating the risk of acquiring these infections?",
             421: "What are the symptoms and complications associated with chlamydial infection in men, and what treatment approaches are recommended? How significant is contact-tracing in managing and preventing the spread of chlamydia among sexual partners?",
             591: "Can you elaborate on the various anti-arrhythmic drugs mentioned in the text, including the administration and specific uses of atropine sulphate, adenosine, and digoxin, and discuss their mechanisms of action, potential side effects, and particular considerations for their use in different patient populations, such as those with severe bradycardia, supraventricular tachycardia (SVT), atrial fibrillation (AF), or renal impairment? Additionally, how are external defibrillation and electrical cardioversion utilized in the treatment of arrhythmias, and what are the procedural steps and precautions involved in these therapeutic interventions?",
             614: "How does an exercise tolerance test approximately 4 weeks after an acute coronary syndrome help in identifying individuals with residual myocardial ischemia and what is the significance of the results of this test in predicting future adverse events within the next 12 months?",
             635: "What are the clinical features and management strategies for mitral regurgitation, and how does the condition progress from mild forms, where the valve remains competent but bulges back into the atrium during systole, to severe cases that may require surgical intervention?",
             773: "What are the key investigations and management strategies for primary amenorrhoea? How does secondary amenorrhoea differ in terms of causes and diagnostic approach, particularly in distinguishing between ovarian dysfunction and hypothalamic/pituitary disorders?",
             810: "What are the challenges in imaging and diagnosing aldosterone-producing adenomas (APAs) using CT or MRI, and what alternative diagnostic procedures are recommended if imaging is inconclusive but there is strong biochemical evidence of an APA? Additionally, what are the management options and their effectiveness for patients with primary hyperaldosteronism, particularly those with an APA?",
             930: "What are the key symptoms and complications of diverticular disease, and how does it resemble conditions like colorectal cancer and inflammatory bowel disease? Explain the diagnostic methods used to identify diverticular disease and its complications. Additionally, outline the management strategies for asymptomatic cases and acute diverticulitis, including dietary recommendations, antibiotic treatment, and surgical options, considering recurrence risks and serious outcomes such as perforation or abscess formation.",
             970: "What are the key features of hepatitis C infection, including its asymptomatic nature and diagnostic challenges? How does the treatment of chronic hepatitis C differ based on viral genotype, and what are the goals and potential complications associated with therapy, including liver transplantation outcomes?",
             1031:"What are the key principles guiding the use of anticoagulant and antithrombotic medications, and how do these drugs differ in their efficacy for preventing arterial versus venous thromboembolism? How are newer oral anticoagulants like dabigatran and rivaroxaban being evaluated in clinical trials for preventing thrombosis, particularly in high-risk orthopedic surgery?",
             1051:"How does the Philadelphia chromosome contribute to the pathogenesis of chronic myeloid leukaemia (CML), and what role does the BCR ABL chimeric gene play in the disease's development, including its impact on cellular processes like proliferation, differentiation, and survival?"}
questionTokens = bertTokenizer(questions[58], padding=True, truncation=True, return_tensors="pt", max_length=512)
questionOutput = bertModel(**questionTokens)
questionEmbedding = torch.mean(questionOutput.last_hidden_state[0], dim=0, dtype=float)
for i in range(dataset.shape[0]):
  # Apply the function to each element in the list and store the results with their indices
    results = [(i, calculate_similarity(x,questionEmbedding)) for i, x in enumerate(tensor_data)]
    
    # Sort the results by the function's output values in descending order and get the top five
    top_five = sorted(results, key=lambda x: x[1], reverse=True)[:5]
    
    # Extract the indices of the top five results
    top_five_indices = [x[0] for x in top_five]
#print(dataset['answer'].tolist()[closestidx])


KeyboardInterrupt: 

In [ ]:
print()

In [ ]:
print(top_five_indices)

[626, 630, 625, 636, 640]
